Import data and packages

In [ ]:
import pandas as pd
from apy import *


# Load the data - only change the path and then use extension Data Wrangler to modify the data
df = pd.read_csv(
eu_15min
)

#df.columns.values[0] = "datetime"

df.head()




Specific for forex datetime index for panda

In [ ]:
def prepare_dataframe_for_backtrader(data):
    """
    Prepares a Pandas DataFrame for use with bt.feeds.PandasData in Backtrader.
    Args:
        data (pd.DataFrame): The input DataFrame with OHLC data.
    Returns:
        pd.DataFrame: A DataFrame ready for Backtrader.
    """
    # Print column names for debugging
    print("Original columns:", data.columns)

    data.rename(columns={
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close/last': 'Close',
        'volume': 'Volume'
    }, inplace=True)

    # Ensure the 'volume' column exists in the data
    if 'Volume' not in data.columns:
        print("Adding a 'volume' column with default values (0)...")
        data['Volume'] = 0  # Add a volume column with default values

    # Ensure 'datetime' column exists
    if 'datetime' not in data.columns:
        raise KeyError("The DataFrame does not have a 'datetime' column.")

    # Ensure 'datetime' column is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(data['datetime']):
        print("Converting 'datetime' column to datetime format...")
        data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')

    # Check for invalid datetime entries
    if data['datetime'].isnull().any():
        raise ValueError("The 'datetime' column contains invalid or missing values after conversion.")

    # Set 'datetime' as the index
    data.set_index('datetime', inplace=True)

    # Ensure the index is sorted (Backtrader requires chronological order)
    if not data.index.is_monotonic_increasing:
        print("Sorting the DataFrame by the 'datetime' index...")
        data.sort_index(inplace=True)

    # Ensure required columns exist
    required_columns = ['Open', 'High', 'Low', 'Close']
    for col in required_columns:
        if col not in data.columns:
            raise ValueError(f"Missing required column: '{col}'")

    print("The DataFrame is ready for Backtrader.")
    return data

# Example usage
clean_data = prepare_dataframe_for_backtrader(df)
clean_data.head()

clean_data.to_csv("Clean_gold_daily_backtrader") #ex: Clean_gold_daily_backtrader